# 📉 Finetuning Our Model for Better Results

In our previous notebook we [built a simple fashion search engine using Docarray](https://colab.research.google.com/github/alexcg1/neural-search-notebooks/blob/main/fashion-search/1_build_basic_search/basic_search.ipynb).

Now we'll finetune our model using [Jina Finetuner](https://finetuner.jina.ai) to deliver better results!

### The skinny on Jina Finetuner

Finetuner lets you tune the weights of any deep neural network for better embeddings on search tasks. It accompanies [Jina](https://github.com/jina-ai/jina) to deliver the last mile of performance for domain-specific neural search applications.

🎛 **Designed for finetuning**: a human-in-the-loop deep learning tool for leveling up your pretrained models in domain-specific neural search applications.

🔱 **Powerful yet intuitive**: all you need is finetuner.fit() - a one-liner that unlocks rich features such as siamese/triplet network, metric learning, self-supervised pretraining, layer pruning, weights freezing, dimensionality reduction.

⚛️ **Framework-agnostic**: promise an identical API & user experience on PyTorch, Tensorflow/Keras and PaddlePaddle deep learning backends.

🧈 **[DocArray](https://docarray.jina.ai) integration**: buttery smooth integration with DocArray, reducing the cost of context-switch between experiment and production.

##  1️⃣ Before you start

If you're in Colab, ensure you have GPU selected as runtime. This will speed up processing. You can find it in *Runtime* ▶️ *Change runtime type*

![](https://github.com/alexcg1/neural-search-notebooks/raw/main/fashion-search/2_finetune_model/images/runtime.png)

## ⚙️ Setup

In [29]:
# Check if we're running in Google Colab
try:
    import google.colab
    in_colab = True
except:
    in_colab = False

In [ ]:
!pip install torchvision~=0.11
!pip install finetuner==0.4

In [ ]:
from docarray import Document, DocumentArray

## 🖼️ Load images

This is just the same process we followed in the last notebook

In [ ]:
DATA_DIR = "./data"
DATA_PATH = f"{DATA_DIR}/*.jpg"
MAX_DOCS = 1000

# Toy data - If data dir doesn't exist, we'll get data of ~800 fashion images from here
TOY_DATA_URL = "https://github.com/alexcg1/neural-search-notebooks/blob/main/fashion-search/data.zip?raw=true"

In [ ]:
# Download images if they don't exist
import os

if not os.path.isdir(DATA_DIR) and not os.path.islink(DATA_DIR):
    print(f"Can't find {DATA_DIR}. Downloading toy dataset")
    !wget "$TOY_DATA_URL" -O data.zip
    !unzip -q data.zip # Don't print out every darn filename
    !rm -f data.zip
else:
    print(f"Nothing to download. Using {DATA_DIR} for data")

In [ ]:
docs = DocumentArray.from_files(DATA_PATH, size=MAX_DOCS)
print(f"{len(docs)} Documents in DocumentArray")

In [ ]:
def preproc(doc):
    return (
        doc.load_uri_to_image_tensor(80, 60) # input images are 60x80 px
        .set_image_tensor_normalization()
        .set_image_tensor_channel_axis(-1, 0)
    )


docs.apply(preproc)

## 🧠 Load model

Again, we're playing the same old song, loading a model just like we did last time. Once again we're using trusty old `resnet50`.

In [ ]:
import torchvision

model = torchvision.models.resnet50(pretrained=True)

### See embeddings

Let's take a look at our embeddings to see how good the current (un-finetuned) model is. In Google Colab we need to install some extra libraries.

In [ ]:
import torch

if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

docs.embed(model, device=DEVICE)

In [ ]:
if not in_colab: # Colab needs way too many dependencies installed for this
    docs.plot_embeddings(image_sprites=True, image_source="uri")

As we can see, most items are in more or less the position you'd expect, and are clustered according to type.

⚠️ To continue, stop the notebook (since the embedding animation blocks the script), then continue from this cell with:

- *Runtime* ▶️ *Run after* (in Google Colab)
- *Run* ▶️ *Run selected cell and all below* (In Jupyter Lab)

### Examine layers

Resnet is a classification model. However, we don't want to train the final (classification) layer, but rather the embedding before that. As we can see below, that layer is called `adaptiveavgpool2d_173` for `resnet50`. We'll set this as our `LAYER_NAME` variable which we'll later use in `ft.fit()`.

---

ℹ️ Different models will have different layer names. So if you used `resnet18` (for example), your `LAYER_NAME` would be `adaptiveavgpool2d_67`.

In [ ]:
# Let's look at the layers
import finetuner as ft
ft.display(model, (3, 80, 60))

LAYER_NAME = "adaptiveavgpool2d_173" # second to last layer name

## 📉 Finetune model

Here's where the new stuff kicks in!

We'll:

- Set some basic parameters
- Install a module to see progress
- Finetune our model, focusing on the embedding layer *just* before the classification layer

In [ ]:
# Basic setup
EPOCHS = 6         # higher = more time, better finetuning
BATCH_SIZE = 64    # higher = use more memory

In [ ]:
# See progress bar in notebook
!pip install -q ipywidgets # -q = quiet
import ipywidgets

While tuning, keep an eye on the loss rate. It starts at about 0.36 then falls to about 0.08 after 6 epochs.

In [ ]:
tuned_model = ft.fit(
    model=model,
    train_data=docs,
    loss='TripletLoss',
    epochs=EPOCHS,
    device=DEVICE,
    batch_size=BATCH_SIZE,
    to_embedding_model=True,
    input_size=(3, 80, 60),
    layer_name=LAYER_NAME, # layer before fc as feature extractor
    freeze=False,
)

### See embeddings

Now that we've tuned the model, let's clear out our old embeddings and look at the new ones. 

⚠️ Unfortunately this doesn't play nice in Google Colab, only Jupyter.

In [28]:
if not in_colab:
    for doc in docs:
        doc.embedding = None
    
    docs.embed(model, device=DEVICE)
    docs.plot_embeddings(image_sprites=True, image_source="uri")

NameError: name 'in_colab' is not defined

Now you can see a much starker delineation between different clothing types, showing the model has been tuned effectively.

⚠️ If you can see the embedding animation above, you'll need to stop the notebook (since the embedding animation blocks the script), then continue from this cell with:

- *Runtime* ▶️ *Run after* (in Google Colab)
- *Run* ▶️ *Run selected cell and all below* (In Jupyter Lab)

## 💾 Save model

In [ ]:
torch.save(tuned_model, "tuned-model")

In [ ]:
# If running in Colab, download to local filesystem
if in_colab:
    files.download("tuned-model")

## ⏭️ Next steps

Next we'll:

- Load the tuned model into our original script
- Compare results with the base model